**Tried Modellig with all the features along with BOW, TFIDF,W2V,TFIDF_W2V of reviews_list and dint get good results. Hence retraining Models by considering only the Reviews_list feature**

In [1]:
#mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#importing libraries
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


import tqdm
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.sparse import  hstack,vstack
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV, train_test_split

In [76]:
#Loading the data
dataset=pd.read_csv("/content/drive/MyDrive/my code/zomato.csv")

In [77]:
dataset.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [79]:
#renoving all the columns except rate abd reviews_list column
dataset=dataset.drop(["url",'address','name','online_order','book_table','votes','phone','location','rest_type','dish_liked','cuisines', 'approx_cost(for two people)', 'menu_item','listed_in(type)','listed_in(city)'],axis=1)

In [80]:
dataset.head()

,rate,reviews_list
0,4.1/5,"[('Rated 4.0', 'RATED\n A beautiful place to ..."
1,4.1/5,"[('Rated 4.0', 'RATED\n Had been here for din..."
2,3.8/5,"[('Rated 3.0', ""RATED\n Ambience is not that ..."
3,3.7/5,"[('Rated 4.0', ""RATED\n Great food and proper..."
4,3.8/5,"[('Rated 4.0', 'RATED\n Very good restaurant ..."


**Cleaning and preprocessing both columns**

In [55]:
dataset['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', nan, '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [56]:
#removing '/5' from rate column
def split(x):
  return x.split('/')[0]

In [109]:
dataset['rate']=dataset['rate'].astype(str)
dataset['rate']=dataset['rate'].apply(split)
dataset['rate'].head()

0    4.1
1    4.1
2    3.8
3    3.7
4    3.8
Name: rate, dtype: object

In [110]:
dataset['rate'].unique()

array(['4.1', '3.8', '3.7', '3.6', '4.6', '4.0', '4.2', '3.9', '3.1',
       '3.0', '3.2', '3.3', '2.8', '4.4', '4.3', '0', '2.9', '3.5', 'nan',
       '2.6', '3.8 ', '3.4', '4.5', '2.5', '2.7', '4.7', '2.4', '2.2',
       '2.3', '3.4 ', '3.6 ', '4.8', '3.9 ', '4.2 ', '4.0 ', '4.1 ',
       '3.7 ', '3.1 ', '2.9 ', '3.3 ', '2.8 ', '3.5 ', '2.7 ', '2.5 ',
       '3.2 ', '2.6 ', '4.5 ', '4.3 ', '4.4 ', '4.9', '2.1', '2.0', '1.8',
       '4.6 ', '4.9 ', '3.0 ', '4.8 ', '2.3 ', '4.7 ', '2.4 ', '2.1 ',
       '2.2 ', '2.0 ', '1.8 '], dtype=object)

So rate column looks better now. Let's remove 'NEW' and '-' and make it look more clean.

In [111]:
dataset.replace('NEW',0,inplace = True)
dataset.replace('-',0,inplace = True)

In [112]:
#change datatype of rate column from object to float

dataset['rate']=dataset['rate'].astype(float)

In [113]:
dataset['rate'].unique()

array([4.1, 3.8, 3.7, 3.6, 4.6, 4. , 4.2, 3.9, 3.1, 3. , 3.2, 3.3, 2.8,
       4.4, 4.3, 0. , 2.9, 3.5, nan, 2.6, 3.4, 4.5, 2.5, 2.7, 4.7, 2.4,
       2.2, 2.3, 4.8, 4.9, 2.1, 2. , 1.8])

In [114]:
#filling null values of rate with mean in rate column

dataset['rate'].fillna(dataset['rate'].mean(),inplace=True)

dataset['rate'].isnull().sum()

0

**<h1>Preprocessing the reviews column</h1>**

In [115]:
dataset['reviews_list'].dropna(inplace=True)
dataset['reviews_list'].shape

(51717,)

In [116]:
#changing the review data tyoe from object to string
dataset['reviews_list']=dataset['reviews_list'].astype("str")

In [95]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"Rated", "", phrase)
    phrase = re.sub(r"RATED", "", phrase)
   
    
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [96]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","rated", "\n", "n","nan", "x"])

In [97]:
from tqdm import tqdm

# tqdm is for printing the status bar
def preprocessed_data(text):
  preprocessed_text= []
  for sentance in tqdm(text):
    
    sentance = re.sub(r"http\S+", "", sentance)
    
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_text.append(sentance.strip())
  return preprocessed_text


In [98]:

dataset["reviews_list"]=preprocessed_data(dataset["reviews_list"].values)

100%|██████████| 51717/51717 [01:01<00:00, 842.55it/s] 


**<h1>SPLITTING THE DATA INTO TRAIN AND TEST VALUES</h1>**

In [118]:
y= dataset["rate"].values
X=dataset.drop(["rate"],axis=1)

In [119]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.33, stratify=y)

print(X_train.shape)
print(X_test.shape)

(34650, 1)
(17067, 1)


 **<h2>FEATURIZING THE REVIEW COLUMN</h2>**

In [120]:
data1=X_train["reviews_list"].values



**AVG W2V FOR TRAIN VALUES**

In [122]:
#lets load glove_vector
#make sure need to have the glove_vectors file
import pickle

with open("/content/drive/MyDrive/my code/glove_vectors",'rb') as f:
  model= pickle.load(f)

  glove_words=set(model.keys())

In [123]:

avg_w2v_vectors=[]  #to store avg_w2v in list
for sentence in tqdm(data1):#for each review from preprocessed reviews
  vector=np.zeros(300)#initialising the vector with zero length
  count_word=0#initialising count_word with valid_vector in each review
  for word in z: #each word from review
    if word in glove_words:
      vector+=model[word]
      count_word+=1
  if count_word!=0:
     vector/=count_word
  avg_w2v_vectors.append(vector)


print(type(avg_w2v_vectors))

100%|██████████| 34650/34650 [00:04<00:00, 7046.25it/s]

<class 'list'>


In [124]:
data2=X_test["reviews_list"].values



**AVG W2V FOR TEST VALUES**

In [125]:
avg_w2v_vectors_test=[]  #to store avg_w2v in list
for sentence in tqdm(data2): #for each review from preprocessed reviews
    vector=np.zeros(300)#initialising the vector with zero length
    count_word=0#initialising count_word with valid_vector in each review
    for word in sentence.split(): #each word from review
      if word in glove_words:
        vector+=model[word]
        count_word+=1
    if count_word!=0:
      vector/=count_word
    avg_w2v_vectors_test.append(vector)
print(len(avg_w2v_vectors_test))
print(len(avg_w2v_vectors_test[0]))

100%|██████████| 17067/17067 [00:22<00:00, 760.68it/s]

17067
300


**<h1>MODELLING</h1>**

**Modelling with all features and Avg W2V Vectorizer**

In [ ]:
#computing the  MSE for LinearRegression Model
model_1_w2v=LinearRegression()

model_1_w2v.fit(w2v,y_train)
model_1_train_pred=model_1_w2v.predict(w2v)  #Predictng for train values
model_1_test_pred=model_1_w2v.predict(w2v_test)  #predicting for test values


print("The train_MSE  for LR model",mean_squared_error(y_train,model_1_train_pred))
print("The test_MSE for LR model",mean_squared_error(y_test,model_1_test_pred))

The train_MSE  for LR model 0.6897697205256759
The test_MSE for LR model 0.6974556843828231


In [ ]:
##computing the  MSE for RandomForestRegression Model
model_2_w2v=RandomForestRegressor(n_jobs=-1)
model_2_w2v.fit(avg_w2v_vectors,y_train)

model_2_train_pred=model_2_w2v.predict(w2v)
model_2_test_pred=model_2_w2v.predict(w2v_test)

print("The train_MSE  for RF model",mean_squared_error(y_train,model_2_train_pred))
print("The test_MSE for RF model",mean_squared_error(y_test,model_2_test_pred))

The train_MSE  for RF model 0.31007212963837183
The test_MSE for RF model 0.4549433538702952


In [ ]:
##computing the  MSE for DecisionTreeRegressor Model
model_3_w2v=DecisionTreeRegressor()
model_3_w2v.fit(avg_w2v_vectors,y_train)
model_3_train_pred=model_3_w2v.predict(avg_w2v_vectors)
model_3_test_pred=model_3_w2v.predict(avg_w2v_vectors_test)


print("The train_MSE  for DT model",mean_squared_error(y_train,model_3_train_pred))
print("The test_MSE for DT model",mean_squared_error(y_test,model_3_test_pred))

The train_MSE  for DT model 0.29075608731982183
The test_MSE for DT model 0.5744753560756303


In [ ]:
##computing  MSE for XGBRegressor Model
model_4_tfidf_w2v=XGBRegressor(n_jobs=-1)
model_4_tfidf_w2v.fit(w2v,y_train)
model_4_train_pred=model_4_tfidf_w2v.predict(w2v)
model_4_test_pred=model_4_tfidf_w2v.predict(w2v_test)

print("The train_MSE  for XGB model",mean_squared_error(y_train,model_4_train_pred))
print("The test_MSE for XGB model",mean_squared_error(y_test,model_4_test_pred))
print("The test_RMSE for XGB model",np.sqrt(mean_squared_error(y_test,model_4_test_pred)))

[09:12:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The train_MSE  for XGB model 0.5656186866208778
The test_MSE for XGB model 0.6356352899074608
The test_RMSE for XGB model 0.7972673892161029


In [ ]:
#computing the  MSE for LGBMRegressor Model
model_5_w2v=LGBMRegressor()  
model_5_w2v.fit(w2v,y_train)   
model_5_train_pred=model_5_w2v.predict(w2v)  #Predictng for train values
model_5_test_pred=model_5_w2v.predict(w2v_test)  #predicting for test values



print("The train_MSE  for LGBM model",mean_squared_error(y_train,model_5_train_pred))
print("The test_MSE for LGBM model",mean_squared_error(y_test,model_5_test_pred))

The train_MSE  for LGBM model 0.37206226158740924
The test_MSE for LGBM model 0.49928880121205577


In [ ]:
##computing the  MSE for KNeighborsRegressor Model

model_6_w2v=KNeighborsRegressor()
model_6_w2v.fit(w2v,y_train)
model_6_train_pred=model_6_w2v.predict(w2v)
model_6_test_pred=model_6_w2v.predict(w2v_test)

print("The train_MSE  for KNN model",mean_squared_error(y_train,model_6_train_pred))
print("The test_MSE for KNN model",mean_squared_error(y_test,model_6_test_pred))

The train_MSE  for KNN model 0.4488103192158927
The test_MSE for KNN model 0.5789491445743135


**OBSERVATION:**

After training all the models with **Avg W2V** vectorizer the MSE of test values of RandomForestRegressor is 0.4549 which less compared to other models.

As RandomForest has got MSE of 0.4549 with Avg W2V, so lets do hyperparameter tuning and check for MSE.

**</h1>CONCATINATING ALL THE FEATURES WITH TFIDF AVG W2V VECTORIZER</h1>**

In [ ]:
param={
    "n_estimators":[300,500,1000],
    "max_depth":[3,6]
    }

clf=RandomForestRegressor(n_jobs=-1)
clf1=RandomizedSearchCV(estimator = clf, param_distributions = param, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
clf1.fit(w2v,y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 187.0min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100, n_jobs=-1,
                   

In [ ]:
best_model=clf1.best_estimator_

In [ ]:
best_model.fit(w2v,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:


best_model_train_pred=best_model.predict(w2v)
best_model_test_pred=best_model.predict(w2v_test)


print("The train_MSE  for Best model",mean_squared_error(y_train,best_model_train_pred))
print("The test_MSE for best model",mean_squared_error(y_test,best_model_test_pred))


The train_MSE  for Best model 0.6072221812162645
The test_MSE for best model 0.6337878324968772


**HyperParameter Tuning with XGBRegressor**

In [ ]:
model=XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, colsample_bytree=0.8)
model.fit(w2v,y_train)
model_train_pred=model.predict(w2v)
model_test_pred=model.predict(w2v_test)
print("The train_MSE  for Best model",mean_squared_error(y_train,model_train_pred))
print("The test_MSE for best model",mean_squared_error(y_test,model_test_pred))

[17:34:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The train_MSE  for Best model 0.2901783833287788
The test_MSE for best model 0.4307734728828007


In [ ]:
param={
    "n_estimators":[300,500,1000],
    "max_depth":[3,6]
    }

clf=XGBRegressor(n_jobs=-1)
clf1=RandomizedSearchCV(estimator = clf, param_distributions = param, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
clf1.fit(w2v,y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 182.0min finished


[04:40:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=-1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=0,
                                          reg_lambda=1, scale_pos_weight=1,
                                          seed=None, silent=None, subsample=1,
                                 

In [ ]:
best_model=clf1.best_estimator_

In [ ]:
best_model.fit(w2v,y_train)

[05:31:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
model_train_pred=best_model.predict(w2v)
model_test_pred=best_model.predict(w2v_test)
print("The train_MSE  for Best model",mean_squared_error(y_train,model_train_pred))
print("The test_MSE for best model",mean_squared_error(y_test,model_test_pred))

The train_MSE  for Best model 0.26182807974950756
The test_MSE for best model 0.35610080174071446
